Notebook that transforms the data from pickle format to a dataframe for easier handling when enriching AIS-data with weather data.

In [29]:
import pandas as pd
import pickle as pkl
import numpy as np

In [30]:
LAT, LON, SOG, COG, TS, MMSI, TID = np.arange(7)

## STAD Publication West Coast Spatial Bounds

In [31]:
# Based on publication source code (see cell later)
wc_lonmin = -126
wc_lonmax = -122
wc_latmin = 34.5
wc_latmax = 38.5
wc_sogmax = 30.0
wc_cogmax = 360.0

# Methods

In [32]:
def unscale_vect(vect, x_min, x_max):
    return vect * (x_max - x_min) + x_min

In [33]:
def stadset_to_df(stad_data,
                  lonmin,
                  lonmax,
                  latmin,
                  latmax,
                  sogmax,
                  cogmax):

    """
    Function to inversely scale the training data in (Li et al., 2024)
    Based on their methodology in: https://github.com/lihui91/ais_process/blob/7f62ff2eb5072ceee723e48f91b0d1eac9218429/src/dataset_preprocessing.py#L325
    """

    colnames = [
        'latitude',
        'longitude',
        'SOG',
        'COG',
        'timestamp',
        'MMSI'
    ]

    # Unscaling
    dfs = list()
    for i, trajectory_data in enumerate(stad_data):

        mmsi = trajectory_data['mmsi']
        trj = trajectory_data['traj']

        trj_id = f"{int(mmsi)}_{i}"

        trj[:, LAT] = unscale_vect(trj[:, LAT], latmin, latmax)
        trj[:, LON] = unscale_vect(trj[:, LON], lonmin, lonmax)
        trj[:, SOG] = trj[:, SOG] * sogmax
        trj[:, COG] = trj[:, COG] * cogmax

        # Transform to dataframe
        df = pd.DataFrame(trj, columns=colnames)
        df['traj_id'] = trj_id
        dfs.append(df)

    df = pd.concat(dfs, axis=0)

    df['MMSI'] = df['MMSI'].astype(int)
    df['timestamp'] = df['timestamp'].astype(int)

    return df

# Data Loading
Dataset was retrieved from https://github.com/lihui91/ais_process

## Original Train

In [ ]:
train_set = '../../data/ct_train.pkl'
with open(train_set, 'rb') as f:
    train = pkl.load(f)

In [35]:
train[0]['traj'][0, :]

array([4.12870000e-01, 8.62255000e-01, 4.20000000e-01, 9.17777778e-01,
       1.67253120e+09, 4.16497000e+08])

## Original Valid

In [ ]:
valid_set = '../../data/ct_valid.pkl'
with open(valid_set, 'rb') as f:
    valid = pkl.load(f)

In [37]:
valid[0]['traj'][0, :]

array([4.72507500e-01, 7.37727500e-01, 3.53333333e-01, 9.25833333e-01,
       1.69084800e+09, 2.59157000e+08])

In [38]:
df_valid = pd.DataFrame(np.vstack([trj['traj'] for trj in valid]), columns=['latitude', 'longitude', 'SOG', 'COG', 'timestamp', 'MMSI'])
df_valid.describe()

,latitude,longitude,SOG,COG,timestamp,MMSI
count,94370.000000,94370.000000,94370.000000,94370.000000,9.437000e+04,9.437000e+04
mean,0.614819,0.715759,0.274238,0.546148,1.692203e+09,4.056864e+08
std,0.262026,0.224070,0.194626,0.280503,7.648862e+05,1.299013e+08
min,0.000008,0.000010,0.000000,0.000000,1.690848e+09,2.056570e+08
25%,0.422599,0.599907,0.045260,0.336111,1.691550e+09,3.110001e+08
50%,0.703124,0.768908,0.321415,0.495300,1.692191e+09,3.691420e+08
75%,0.824240,0.908844,0.423333,0.823217,1.692837e+09,5.380075e+08
max,0.999997,0.999997,0.790108,1.000000,1.693526e+09,6.360930e+08


# Unscale and transform to DataFrame

In [39]:
stad_train_df = stadset_to_df(train,
                              wc_lonmin,
                              wc_lonmax,
                              wc_latmin,
                              wc_latmax,
                              wc_sogmax,
                              wc_cogmax)
print(stad_train_df.shape)
stad_train_df.head()

(538223, 7)


,latitude,longitude,SOG,COG,timestamp,MMSI,traj_id
0,36.151480,-122.550980,12.600000,330.400000,1672531202,416497000,416497000_0
1,36.180222,-122.573689,12.200000,327.474286,1672531802,416497000,416497000_0
2,36.208445,-122.596052,12.265217,328.300000,1672532402,416497000,416497000_0
3,36.236860,-122.619197,12.405714,326.837143,1672533002,416497000,416497000_0
4,36.264735,-122.644051,12.322857,322.640000,1672533602,416497000,416497000_0


In [40]:
stad_train_df.describe()

,latitude,longitude,SOG,COG,timestamp,MMSI
count,538223.000000,538223.000000,538223.000000,538223.000000,5.382230e+05,5.382230e+05
mean,37.025807,-123.082414,8.110445,199.766808,1.682567e+09,4.051916e+08
std,0.977095,0.861746,5.822389,103.043849,5.337238e+06,1.252096e+08
min,34.500010,-125.999980,0.000000,0.000000,1.672531e+09,2.055590e+08
25%,36.374493,-123.548966,1.515595,121.536149,1.677991e+09,3.110009e+08
50%,37.335614,-122.835558,9.679710,179.000000,1.683208e+09,3.690400e+08
75%,37.794796,-122.360010,12.577049,304.700000,1.687362e+09,5.380061e+08
max,38.499980,-122.000010,24.100000,360.000000,1.690848e+09,6.421220e+08


In [41]:
stad_valid_df = stadset_to_df(valid,
                              wc_lonmin,
                              wc_lonmax,
                              wc_latmin,
                              wc_latmax,
                              wc_sogmax,
                              wc_cogmax)
print(stad_valid_df.shape)
stad_valid_df.head()

(94370, 7)


,latitude,longitude,SOG,COG,timestamp,MMSI,traj_id
0,36.390030,-123.049090,10.600000,333.300000,1690848001,259157000,259157000_0
1,36.416742,-123.065319,10.756338,331.398592,1690848601,259157000,259157000_0
2,36.443054,-123.083440,10.528571,330.657143,1690849201,259157000,259157000_0
3,36.468740,-123.101660,10.400000,330.385915,1690849801,259157000,259157000_0
4,36.494224,-123.119153,10.357143,331.585714,1690850401,259157000,259157000_0


# Save as parquet files

In [ ]:
stad_train_df.to_parquet('../../data/df_stad_train_west.parquet')
stad_valid_df.to_parquet('../../data/df_stad_valid_west.parquet')